In [70]:
import tensorflow as tf

In [78]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
import os

batch_size = 1000
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [72]:
print(x_train.shape[1:])
print(type(x_train))

(32, 32, 3)
<class 'numpy.ndarray'>


In [73]:
import numpy as np

In [79]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    gray = np.expand_dims(gray, axis=3)
    print(gray.shape)
    return gray

In [80]:
x_train=rgb2gray(x_train)
x_test=rgb2gray(x_test)

(50000, 32, 32, 1)
(10000, 32, 32, 1)


In [81]:
print(x_train[0].shape)

(32, 32, 1)


In [82]:
print(x_test.shape)

(10000, 32, 32, 1)


In [83]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [86]:
model = Sequential()
model.add(Convolution2D(6, (5, 5), padding='valid', data_format="channels_last", input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))   
model.add(Convolution2D(16, (5, 5), padding='valid', data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [87]:
print(x_train[0].shape)
print(x_train[0,:,:,:])

(32, 32, 1)
[[[ 61.2111]
  [ 44.9847]
  [ 48.023 ]
  ...
  [137.0222]
  [130.4358]
  [128.7672]]

 [[ 18.8024]
  [  0.    ]
  [ 10.0762]
  ...
  [ 94.6907]
  [ 89.9901]
  [ 94.0328]]

 [[ 23.9545]
  [  8.8914]
  [ 31.4071]
  ...
  [ 90.2782]
  [ 90.876 ]
  [ 80.2191]]

 ...

 [[172.9052]
  [153.7659]
  [156.6532]
  ...
  [133.875 ]
  [ 35.7334]
  [ 38.0797]]

 [[146.339 ]
  [128.6987]
  [143.5434]
  ...
  [152.5896]
  [ 69.2633]
  [ 59.7957]]

 [[150.6573]
  [136.6542]
  [146.7751]
  ...
  [188.5304]
  [123.9759]
  [ 98.9767]]]


In [88]:
y_test.shape[1]

10

In [89]:
# Loss function
def neg_log_likelihood_loss(y_true, y_pred):
    ##sep = y_pred.shape[1] // 2
    mu, logvar = y_pred[:, :10], y_pred[:, 10:]
    return K.sum(0.5*(logvar+np.log(2*np.pi)+K.square((y_true-mu)/K.exp(0.5*logvar))), axis=-1)

In [90]:
# initiate RMSprop optimizer
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['categorical_crossentropy', 'accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [91]:
type(x_train)

numpy.ndarray

In [92]:
# model.fit(x_train, y_train,
#          batch_size=batch_size,
#          nb_epoch=epochs,
#          #validation_data=(x_test, y_test),
#          shuffle=True)

In [93]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/10
50/50 [==============================] - 7s 148ms/step - loss: 2.2344 - categorical_crossentropy: 2.2344 - accuracy: 0.1636 - val_loss: 2.0740 - val_categorical_crossentropy: 2.0740 - val_accuracy: 0.2562
Epoch 2/10
50/50 [==============================] - 7s 146ms/step - loss: 2.0700 - categorical_crossentropy: 2.0700 - accuracy: 0.2510 - val_loss: 1.8790 - val_categorical_crossentropy: 1.8790 - val_accuracy: 0.3225
Epoch 3/10
50/50 [==============================] - 7s 147ms/step - loss: 1.9323 - categorical_crossentropy: 1.9323 - accuracy: 0.3065 - val_loss: 1.8027 - val_categorical_crossentropy: 1.8027 - val_accuracy: 0.3555
Epoch 4/10
50/50 [==============================] - 7s 148ms/step - loss: 1.9586 - categorical_crossentropy: 1.9586 - accuracy: 0.2990 - val_loss: 1.8626 - val_categorical_crossentropy: 1.8626 - val_accuracy: 0.3335
Epoch 5/10
50/50 [==============================] - 7s 148ms/step - loss: 1.8742 - categorical_crosse

In [94]:
# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)
print('Model Accuracy = %.2f, Model loss = %.2f ' % (evaluation[2], evaluation[1]))

Instructions for updating:
Please use Model.evaluate, which supports generators.
Model Accuracy = 0.36, Model loss = 1.80 


In [95]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
##model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
#scores = model.evaluate(x_test, y_test,show_accuracy=True, verbose=1, batch_size=1000)
scores = model.evaluate(x_test, y_test, verbose=1, batch_size=2000)
print(scores)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])

Saved trained model at /root/shared/saved_models/keras_cifar10_trained_model.h5 
5/5 [==============================] - 0s 4ms/step - loss: 1.7405 - categorical_crossentropy: 1.7405 - accuracy: 0.3806
[1.7404773235321045, 1.7404773235321045, 0.3806000053882599]


In [77]:
print(batch_size)

64
